In [1]:
import pandas as pd
from datetime import datetime

import sqlite3
from sqlalchemy import create_engine

Let's write few functions that will help us interacting with SQLite databse.

In [2]:
def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: # You can optionally pass drop_table_name to drop the table. 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def insert_sql_statement(insert_data, conn, table_name):
    with conn:
        cur = conn.cursor()

        empty_value_container = "?,"*len(insert_data[0])
        empty_value_container = "".join(["(",empty_value_container.strip(","),")"])

        cur.executemany(f"INSERT INTO {table_name} VALUES {empty_value_container}", insert_data)
        
def read_sql_query(query, conn):
    result = pd.read_sql_query(query, conn)
    return result

Let's first create connection to database

In [3]:
database_filename = "info.db"
conn = create_connection(database_filename)

Let's now create database relations

In [4]:
def create_users_table():
    
    insert_data = list(zip([1,2,3,4,5,6],list(map(lambda date: datetime.strptime(date, "%m/%d/%Y"), ["01/01/2001", "01/01/2001", "01/01/2001", "02/01/2001", "02/01/2001", "02/01/2001"]))))

    sql_create_statement = """

                CREATE Table users(
                    user_id Integer not null Primary key,
                    created_at date
                )

    """

    create_table(conn, sql_create_statement, drop_table_name="users")

    insert_sql_statement(insert_data, conn, "users")
    
create_users_table()

In [5]:
def create_exercises_table():
    
    insert_data = list(zip(["e1","e2","e3","e4","e5","e6","e7","e8","e9"], [1,2,3,4,5,5,6,6,6], list(map(lambda date: datetime.strptime(date, "%m/%d/%Y"), ["01/20/2001","01/21/2001","12/20/2001", "02/20/2001", "02/21/2001", "02/21/2001", "02/22/2001", "02/23/2001", "02/24/2001"]))))

    sql_create_statement = """

                create table exercises(
                    exercise_id str not null unique,
                    user_id integer,
                    exercise_completion_date date,
                    
                    foreign key(user_id) references users
                )

    """

    create_table(conn, sql_create_statement, drop_table_name="exercises")

    insert_sql_statement(insert_data, conn, "exercises")
    
create_exercises_table()

In [6]:
def create_providers_table():
    
    insert_data = list(zip(["pr1", "pr2", "pr3", "pr4", "pr5", "pr6", "pr7", "pr8", "pr9"], ["o1", "o1", "o2", "o2", "o3", "o3", "o4", "o5", "o6"], ["Org1", "Org1", "Org2", "Org2", "Org3", "Org3", "Org4", "Org5", "Org6"]))

    sql_create_statement = """

                create table providers(
                    provider_id str not null Primary key,
                    organization_id str not null,
                    organization_name str
                )

    """

    create_table(conn, sql_create_statement, drop_table_name="providers")

    insert_sql_statement(insert_data, conn, "providers")
    
create_providers_table()

In [7]:
def create_phq9_table():
    
    insert_data = list(zip(["p1", "p2", "p3", "p4", "p5", "p6", "p7", "p8", "p9", "p10", "p11", "p12", "p13", "p14"], ["pr1", "pr1", "pr2", "pr2", "pr3", "pr4", "pr4", "pr5", "pr5", "pr6", "pr6", "pr7", "pr8", "pr9"], [12, 23, 20, 22, 15, 16, 18, 20, 24, 21, 27, 13, 15, 16], list(map(lambda date: datetime.strptime(date, "%m/%d/%Y"), ["01/01/2020", "01/02/2020", "01/03/2020", "01/04/2020", "01/05/2020", "01/06/2020", "01/07/2020", "01/08/2020", "01/09/2020", "01/10/2020", "01/11/2020", "01/12/2020", "01/13/2020", "01/14/2020"]))))

    sql_create_statement = """

                create table phq9(
                    patient_id str Primary key,
                    provider_id str not null,
                    score integer,
                    datetime_created date,
                    
                    foreign key(provider_id) references providers
                )

    """

    create_table(conn, sql_create_statement, drop_table_name="phq9")

    insert_sql_statement(insert_data, conn, "phq9")
    
create_phq9_table()

Check any database relation we created!

In [8]:
query_statement = """

    select * from exercises

"""
read_sql_query(query_statement, conn)

,exercise_id,user_id,exercise_completion_date
0,e1,1,2001-01-20 00:00:00
1,e2,2,2001-01-21 00:00:00
2,e3,3,2001-12-20 00:00:00
3,e4,4,2001-02-20 00:00:00
4,e5,5,2001-02-21 00:00:00
5,e6,5,2001-02-21 00:00:00
6,e7,6,2001-02-22 00:00:00
7,e8,6,2001-02-23 00:00:00
8,e9,6,2001-02-24 00:00:00


### Solution 1:

In [9]:
query_statement = """

        select strftime('%m', users.created_at) as month, (cast(count(distinct(users.user_id)) as REAL)/for_month.cohort_size)*100 as user_perc from users
        
        join exercises on (users.user_id = exercises.user_id) and (strftime('%m', users.created_at) = strftime('%m', exercises.exercise_completion_date))
        
        join (select strftime('%m', created_at) as month, count(*) cohort_size from users group by month) as for_month
        
        on strftime('%m', users.created_at) = for_month.month
        
        group by month
        

"""

In [10]:
read_sql_query(query_statement, conn)

,month,user_perc
0,01,66.666667
1,02,100.000000


### Solution 2:

In [11]:
query = """

    select exercises.user_id, count(*) num_exec_comp from exercises
    where exercise_completion_date is not null
    group by exercises.user_id

"""

In [12]:
pd.read_sql_query(query, conn)

,user_id,num_exec_comp
0,1,1
1,2,1
2,3,1
3,4,1
4,5,2
5,6,3


### Solution 3:

In [13]:
query = """

       select organization_id, avg(score) avg_score from phq9
       join providers on phq9.provider_id = providers.provider_id
       group by organization_id
       order by avg_score desc
       limit 5

"""

In [14]:
pd.read_sql_query(query, conn)

,organization_id,avg_score
0,o3,23.000000
1,o1,19.250000
2,o2,16.333333
3,o6,16.000000
4,o5,15.000000
